In [8]:
import pandas as pd

# Đọc file gốc
excel_path = "data_cleaned.xlsx"
df = pd.read_excel(excel_path, sheet_name= 'customer')

In [9]:
df['DOB'] = pd.to_datetime(df['DOB'], origin='1899-12-30', unit='D', errors='coerce')

In [10]:
print(df[['DOB']].head())

         DOB
0 1999-04-18
1 1998-04-16
2 1993-12-16
3 1999-11-08
4 1994-10-01


In [11]:
df.isnull().sum() 

customerid          0
DOB                 0
gender              0
address             1
job                 0
industry         1122
year_of_birth       0
dtype: int64

In [ ]:
string_columns = ['gender', 'address','website', 'job', 'industry']
for col in string_columns:
    df[col] = df[col].astype(str).str.strip().str.lower()

In [ ]:
df=df.drop(columns=['Website'])

In [15]:
df['year_of_birth'] = df['DOB'].dt.year

In [16]:
#Tìm năm sinh phổ biến nhất theo job
most_common_year_by_job = {}
for job in df['job'].dropna().unique():
    subset = df[(df['job'] == job) & (df['year_of_birth'].notna())]['year_of_birth']
    if not subset.empty:
        mode_year = subset.mode()
        if not mode_year.empty:
            most_common_year_by_job[job] = int(mode_year[0])

In [17]:
#Hàm điền DOB bị thiếu
def fill_missing_dob(row):
    if pd.isna(row['DOB']):
        job = row['job']
        if job in most_common_year_by_job:
            year = most_common_year_by_job[job]
            return pd.Timestamp(f'{year}-01-01')
    return row['DOB']

In [18]:
# 8. Áp dụng hàm để điền DOB
df['DOB'] = df.apply(fill_missing_dob, axis=1)

In [19]:
# 9. Cập nhật lại year_of_birth sau khi đã điền thêm DOB
df['year_of_birth'] = df['DOB'].dt.year

In [20]:
df.isnull().sum()

customerid       0
DOB              0
gender           0
address          0
job              0
industry         0
year_of_birth    0
dtype: int64

In [21]:
film = pd.read_excel(excel_path, sheet_name = "film")

In [22]:
film.isnull().sum()

show_id          0
title            0
director        10
cast             6
country          7
release_year     0
rating           1
duration         0
listed_in        0
description      0
dtype: int64

In [23]:
# làm sạch dữ liệu cột desciption cà cast của film
from ftfy import fix_text

def clean_text(text):
    if isinstance(text, str):
        text = fix_text(text)  # Sửa lỗi mã hóa
        for ch in ['“', '”', '"', "'"]:
            text = text.replace(ch, '')  # Bỏ dấu ngoặc kép
        return text
    return text

# Áp dụng cho các cột
film['description'] = film['description'].apply(clean_text)
film['cast'] = film['cast'].apply(clean_text)


!pip install ftfy

In [24]:
film[['cast', 'description']].head(20)

,cast,description
0,"James McAvoy, Michael Fassbender, Jennifer Law...",When Jean Grey transforms into the Dark Phoeni...
1,"Louis Ashbourne Serkis, Tom Taylor, Rebecca Fe...","When a kid discovers the legendary sword, Exca..."
2,"a-chan , KASHIYUKA , NOCCHi",J-Pop band Perfume shares their passion for mu...
3,YOSHIKI,Yoshiki from X Japan performs two Disney songs...
4,Dan Nachtrab,Great Shark Chow Down – prepare for a feast of...
5,"Jeremiah Sullivan, Dave Hoffman",Marine biologist attempts to get bitten by a d...
6,Bert Morris,"Discover the technical prowess behind Petra, a..."
7,Jay Sanders,"National Geographic reconstructs the Ulfberht,..."
8,Maite Jauregui,Forensic experts scan Pompeiis victims to inve...
9,Loïck Peyron,Loïck Peyron investigates the 1978 Amoco Cadiz...


In [ ]:
output_path = r"C:\data _cinema\notebook\data_cleaned.xlsx"
readme = pd.read_excel(excel_path, sheet_name = "readme")
ticket = pd.read_excel(excel_path, sheet_name = "ticket")

# Đọc lại toàn bộ sheet
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    readme.to_excel(writer, sheet_name='readme', index=False)
    df.to_excel(writer, sheet_name='customer', index=False)
    ticket.to_excel(writer, sheet_name='ticket', index=False)
    film.to_excel(writer, sheet_name='film', index=False)

In [27]:
# -*- coding: utf-8 -*-
import re, unicodedata, difflib, pandas as pd
import shutil

# ====== CẤU HÌNH ======
excel_path = "data_cleaned.xlsx"
src_sheet  = "customer"          # sheet chứa cột address
addr_col   = "address"           # tên cột địa chỉ
DEFAULT_CITY = "Đà Nẵng"
OVERWRITE_BACKUP = True          # backup file gốc

if OVERWRITE_BACKUP:
    shutil.copyfile(excel_path, excel_path + ".bak")

# ==== 0) Utils ====
def strip_accents(s: str) -> str:
    s = unicodedata.normalize('NFKD', str(s))
    s = ''.join(c for c in s if not unicodedata.combining(c))
    return s.replace('Đ','D').replace('đ','d')

def norm_key(s: str) -> str:
    s = strip_accents(str(s)).lower()
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def best_match(s, candidates, cutoff=0.86):
    if not s: return (None, 0.0)
    key = norm_key(s)
    keys = list(candidates.keys())
    matches = difflib.get_close_matches(key, keys, n=1, cutoff=cutoff)
    if matches:
        mk = matches[0]
        score = difflib.SequenceMatcher(None, key, mk).ratio()
        return (candidates[mk], score)
    return (None, 0.0)

def clean_street_name(name: str) -> str:
    # loại tiền tố Đường/Duong/Đg...
    return re.sub(r'^(đường|duong|đg\.?|dg\.?)\s+', '', str(name), flags=re.I).strip()

# ==== 1) ĐIỀN THỦ CÔNG TẠI ĐÂY ====
# 👉 Bạn tự thêm/sửa danh sách phường & tên đường cho từng quận.
#    Mẫu đúng: mỗi quận là 1 key "Q. Tên Quận": [ "Tên 1", "Tên 2", ... ]

WARDS_BY_DISTRICT = {
    "Q. Hải Châu":   ["Thuận Phước","Thạch Thang","Thanh Bình","Hải Châu I","Hải Châu II",
        "Phước Ninh","Hòa Thuận Tây","Hòa Thuận Đông","Nam Dương",
        "Bình Hiên","Bình Thuận","Hòa Cường Bắc","Hòa Cường Nam"],        # ví dụ
    "Q. Thanh Khê":  ["An Khê","Chính Gián","Hòa Khê","Tam Thuận","Tân Chính",
        "Thạc Gián","Thanh Khê Đông","Thanh Khê Tây","Vĩnh Trung","Xuân Hà"],
    "Q. Sơn Trà":    ["An Hải Bắc","An Hải Đông","An Hải Tây","Mân Thái",
                        "Nại Hiên Đông","Phước Mỹ","Thọ Quang"],
    "Q. Ngũ Hành Sơn": ["Hòa Hải","Hòa Quý","Khuê Mỹ","Mỹ An"],
    "Q. Liên Chiểu": ["Hòa Hiệp Bắc","Hòa Hiệp Nam","Hòa Khánh Bắc","Hòa Khánh Nam","Hòa Minh", "Hòa Khánh"],
    "Q. Cẩm Lệ":     ["Hòa An","Hòa Phát","Hòa Thọ Đông","Hòa Thọ Tây","Hòa Xuân","Khuê Trung"],
}

STREETS_BY_DISTRICT = {
    "Q. Hải Châu": [
        "Lê Duẩn","Quang Trung","Phan Đình Phùng","Trần Phú","Ông Ích Khiêm",
        "Phan Châu Trinh","Lê Đình Dương","Yên Bái","Nguyễn Chí Thanh",
        "Hải Phòng","Hùng Vương","Lý Tự Trọng","Thái Phiên","Pasteur",
        "Phạm Hồng Thái","Triệu Nữ Vương","Bạch Đằng","Duy Tân","2 Tháng 9",
        "Phạm Ngũ Lão","Thanh Sơn", "Lê Duẩn","Quang Trung","Phan Đình Phùng","Trần Phú","Ông Ích Khiêm",
        "Phan Châu Trinh","Lê Đình Dương","Yên Bái","Nguyễn Chí Thanh",
        "Hải Phòng","Hùng Vương","Lý Tự Trọng","Thái Phiên","Pasteur",
        "Phạm Hồng Thái","Triệu Nữ Vương","Bạch Đằng","Duy Tân","2 Tháng 9",
        "Phạm Ngũ Lão","Thanh Sơn", "Nguyễn Văn Linh","Hoàng Diệu","Phan Bội Châu","Trần Quốc Toản","Lê Lợi","Nguyễn Du","Đống Đa",
        "Chu Văn An","Huỳnh Thúc Kháng","Ngô Gia Tự","Lý Thường Kiệt","Nguyễn Trãi",
        "Hồ Tùng Mậu","Nguyễn Thành Hãn","Cô Giang","Phạm Đình Hổ","Yên Bái",
        "Cao Thắng","Đào Duy Từ","Bùi Viện","Trần Bình Trọng"
    ],
    "Q. Thanh Khê": [
        "Điện Biên Phủ","Thái Thị Bôi","Lê Độ","Nguyễn Phước Nguyên","Trần Cao Vân",
        "Nguyễn Tri Phương","Trưng Nữ Vương","Nguyễn Hoàng","Hà Huy Tập","Hàm Nghi",
        "Cù Chính Lan","Phan Thanh","Huỳnh Ngọc Huệ","Trần Xuân Lê","Đỗ Quang",
        "Lý Thái Tổ","Yên Khê", "Hoàng Hoa Thám","Kỳ Đồng","Võ Văn Tần","Phạm Nhữ Tăng",
        "Phạm Văn Nghị","Lê Duy Đình","Nguyễn Văn Huề","Nguyễn Chánh"
    ],
    "Q. Sơn Trà": [
        "Trần Hưng Đạo","Yết Kiêu","Ngô Quyền","Chu Huy Mân","Hồ Nghinh",
        "Hoàng Sa","Võ Văn Kiệt","Nguyễn Công Trứ","Dương Vân Nga","Lê Tấn Trung",
        "Trần Nhân Tông","Phạm Cự Lượng","Phạm Văn Đồng","Giáp Văn Cương",
        "An Đồn","Mân Quang","Lê Hữu Trác","Hoàng Đức Lương",
        "Ngọc Hân","Phan Kế Bính","Phùng Chí Kiên"
    ],
    "Q. Ngũ Hành Sơn": [
        "Lê Văn Hiến","Nguyễn Văn Thoại","Trần Đại Nghĩa","Minh Mạng",
        "Lê Quang Đạo","Hồ Xuân Hương","Nghiêm Xuân Yêm","Võ Chí Công",
        "Trường Sa","Ngũ Hành Sơn","Võ Nguyên Giáp", "Lạc Long Quân","Triệu Việt Vương","Hải Sơn","Phan Huy Ôn",
        "Mai An","Thế Lữ","Vũ Quỳnh"
    ],
    "Q. Liên Chiểu": [
        "Nguyễn Lương Bằng","Tôn Đức Thắng","Hoàng Văn Thái","Nguyễn Tất Thành",
        "Kinh Dương Vương","Nam Cao","Bắc Đẩu","Bàu Tràm",
        "Phạm Như Xương","Âu Cơ","Lê Ngô Cát","Lê Văn Long",
        "Phạm Khu Tiết","Nguyễn Đình Tựu", "Hoàng Tăng Bí","Nguyễn Văn Cừ","Ung Văn Khiêm","Lê Thiệt",
        "Nguyễn Cảnh Dị","Tô Hiệu","Văn Cao","Nguyễn Huy Tưởng"
    ],
    "Q. Cẩm Lệ": [
        "Tôn Đản","Trần Nam Trung","Phạm Hữu Kính","Cách Mạng Tháng 8","Tiểu La",
        "Trường Chinh","Đoàn Nhữ Hài","Đinh Tiên Hoàng","Thăng Long","Phạm Phú Thứ", "Lê Trọng Tấn","Tôn Thất Tùng","Ông Ích Đường","Đỗ Huy Uyển",
        "Nguyễn Nghiêm","Trường Sơn"
    ],
}

# (tuỳ chọn) alias chính tả
WARD_ALIAS = {
    norm_key("Hoa Khe"): "Hòa Khê",
}
STREET_ALIAS = {
    norm_key("Trung Nữ Vương"): "Trưng Nữ Vương",
    norm_key("Duong Nguyen Van Linh"): "Nguyễn Văn Linh",
}

# ==== 2) Sinh map dùng cho matching ====
# District map (quận/huyện/tỉnh lân cận để nhận diện nếu người dùng đã ghi sẵn)
districts_dn = ["Hải Châu","Thanh Khê","Sơn Trà","Ngũ Hành Sơn","Liên Chiểu","Cẩm Lệ","Hòa Vang","Hoàng Sa"]
districts_qn = ["Quế Sơn","Điện Bàn","Duy Xuyên","Thăng Bình","Núi Thành"]

district_map = { norm_key(d): f"Q. {d}" for d in districts_dn if d not in ["Hòa Vang","Hoàng Sa"] }
district_map.update({ norm_key("Hòa Vang"): "H. Hòa Vang", norm_key("Hoàng Sa"): "H. Hoàng Sa" })
district_map.update({ norm_key(d): f"H. {d}" for d in districts_qn })

# Ward maps
ward_map, ward_district_map = {}, {}
for dist, names in WARDS_BY_DISTRICT.items():
    for raw in names:
        name = WARD_ALIAS.get(norm_key(raw), raw)
        k = norm_key(name)
        ward_map[k] = f"P. {name}"
        ward_district_map[k] = dist

# Street maps
street_map, street_district_map = {}, {}
for dist, names in STREETS_BY_DISTRICT.items():
    for raw in names:
        name = STREET_ALIAS.get(norm_key(raw), clean_street_name(raw))
        k = norm_key(name)
        street_map[k] = f"Đường {name}"
        street_district_map.setdefault(k, dist)

# Tên TP viết tắt
city_map = {
    norm_key("Đà Nẵng"): "Đà Nẵng",
    norm_key("DN"): "Đà Nẵng",
    norm_key("Da Nang"): "Đà Nẵng",
    norm_key("Quảng Nam"): "Quảng Nam",
    norm_key("QN"): "Quảng Nam",
}

# ==== 3) Chuẩn hoá 1 địa chỉ ====
def normalize_address(raw: str, cutoff=0.86):
    if pd.isna(raw) or str(raw).strip()=="":
        return {"address_clean": None, "street":None,"ward":None,"district":None,"city":None,"status":"empty"}

    s = str(raw)
    s = unicodedata.normalize('NFC', s).replace('\u00A0',' ')
    s = re.sub(r'\s+', ' ', s).strip()
    key = norm_key(s)

    # Nhận diện TP nhanh
    city = None
    for k, canon in city_map.items():
        if k in key:
            city = canon
            break
    if city is None and re.search(r'(,|\s)\b(dn|đn)\b$', key):
        city = "Đà Nẵng"

    # N-grams 1..3
    tokens = key.split()
    ngrams = set(' '.join(tokens[i:i+n]) for n in (1,2,3) for i in range(len(tokens)-n+1))

    # Fuzzy match
    w_best = max((best_match(g, ward_map, cutoff)     for g in ngrams), key=lambda x: x[1], default=(None,0))
    d_best = max((best_match(g, district_map, cutoff) for g in ngrams), key=lambda x: x[1], default=(None,0))
    s_best = max((best_match(g, street_map, cutoff)   for g in ngrams), key=lambda x: x[1], default=(None,0))
    ward, w_score = w_best
    district, d_score = d_best
    street, s_score = s_best

    # Suy luận từ phường
    if city is None and ward:
        city = DEFAULT_CITY
    if district is None and ward:
        if not hasattr(normalize_address, "_ward_key_by_label"):
            normalize_address._ward_key_by_label = {v: k for k, v in ward_map.items()}
        wkey = normalize_address._ward_key_by_label.get(ward)
        if wkey in ward_district_map:
            district = ward_district_map[wkey]

    # Suy luận từ đường
    if city is None and street:
        city = DEFAULT_CITY
    if district is None and street:
        if not hasattr(normalize_address, "_street_key_by_label"):
            normalize_address._street_key_by_label = {v: k for k, v in street_map.items()}
        skey = normalize_address._street_key_by_label.get(street)
        if skey in street_district_map:
            district = street_district_map[skey]

    # Suy city từ loại district
    if city is None:
        if district and district.startswith("Q."):
            city = "Đà Nẵng"
        elif district and district.startswith("H. Quế Sơn"):
            city = "Quảng Nam"

    parts = [p for p in [street, ward, district, city] if p]
    addr_clean = ', '.join(parts) if parts else s.title()

    status = "ok"
    flags = []
    if s_score < cutoff and street:   flags.append("street_low_conf")
    if w_score < cutoff and ward:     flags.append("ward_low_conf")
    if d_score < cutoff and district: flags.append("district_low_conf")
    if not (street or ward or district or city): flags.append("no_match")
    if flags: status = ";".join(flags)

    return {"address_clean": addr_clean, "street": street, "ward": ward,
            "district": district, "city": city, "status": status}

# ==== 4) ÁP DỤNG & GHI FILE ====
sheets = pd.read_excel(excel_path, sheet_name=None)
if src_sheet not in sheets:
    raise KeyError(f"Không tìm thấy sheet '{src_sheet}'.")
df_cus = sheets[src_sheet].copy()

if addr_col not in df_cus.columns:
    raise KeyError(f"Sheet '{src_sheet}' không có cột '{addr_col}'. Columns: {list(df_cus.columns)}")

res = df_cus[addr_col].apply(normalize_address).apply(pd.Series)
df_cus[addr_col] = res["address_clean"]
# df_cus["status"] = res["status"]   # mở nếu muốn soát lỗi

# Xoá cột phụ nếu từng tồn tại
for c in ["address_clean","street","ward","district","city"]:
    if c in df_cus.columns:
        df_cus.drop(columns=[c], inplace=True)

sheets[src_sheet] = df_cus
with pd.ExcelWriter(excel_path, engine="openpyxl") as w:
    for name, dfi in sheets.items():
        dfi.to_excel(w, sheet_name=name, index=False)

print("Done: đã ghi đè cột 'address' trong sheet 'customer'.")
print(df_cus[[addr_col]])

Done: đã ghi đè cột 'address' trong sheet 'customer'.
                                           address
0                P. Hòa Khê, Q. Thanh Khê, Đà Nẵng
1                 P. Mân Thái, Q. Sơn Trà, Đà Nẵng
2                            H. Quế Sơn, Quảng Nam
3           Đường Trường Chinh, Q. Cẩm Lệ, Đà Nẵng
4                               Q. Cẩm Lệ, Đà Nẵng
...                                            ...
4474                                       Đà Nẵng
4475              P. Phước Mỹ, Q. Sơn Trà, Đà Nẵng
4476                             Nguyen Gian Thanh
4477  Đường Ngũ Hành Sơn, Q. Ngũ Hành Sơn, Đà Nẵng
4478       P. Thanh Khê Tây, Q. Thanh Khê, Đà Nẵng

[4479 rows x 1 columns]
